In [22]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizerfrom sklearn.preprocessing import KBinsDiscretizer

# 파일 불러오기
facilities_df = pd.read_csv("행정동별_시설물_개수.csv", encoding='utf-8')
green_df = pd.read_csv("녹지.csv", encoding='utf-8')
pet_ratio_df = pd.read_csv("반려동물수_비율.csv", encoding='euc-kr')
leisure_income_df = pd.read_csv("seoul_leisure_pet_facilities.csv", encoding='utf-8')
dong_area_df = pd.read_csv("dong_area.csv", encoding='utf-8')
area_df = pd.read_csv("면적.csv", encoding='utf-8')


SyntaxError: invalid syntax (25416319.py, line 4)

In [23]:
# 1. '동별(2)'을 '구'로 이름 변경
dong_area_df = dong_area_df.rename(columns={'동별(2)': '구'})

# 2. '구' 기준으로 녹지면적 병합
dong_area_df = dong_area_df.merge(green_df[['구', '녹지총면적(㎡)']], on='구', how='left')
dong_area_df = dong_area_df.rename(columns={'녹지총면적(㎡)': '녹지총면적'})
# 결과 확인
print(dong_area_df.head())

# 선택 사항: 저장
dong_area_df.to_csv("dong_area_with_green.csv", index=False, encoding='utf-8-sig')


  동별(1)    구  행정동      면적  구성비 (%)     행정   법정       소계     소계.1     녹지총면적  \
0   서울시   소계   소계  605.20   100.00  426.0  467  12926.0  96663.0       NaN   
1   서울시  종로구   소계   23.91     3.95   17.0   87    279.0   1475.0  228451.0   
2   서울시  종로구  사직동    1.23     0.20    1.0   12     16.0     68.0  228451.0   
3   서울시  종로구  삼청동    1.49     0.25    1.0    7     10.0     42.0  228451.0   
4   서울시  종로구  부암동    2.27     0.38    1.0    3     18.0     98.0  228451.0   

      녹지총면적  
0       NaN  
1  228451.0  
2  228451.0  
3  228451.0  
4  228451.0  


In [33]:
quarter_cols = [col for col in leisure_income_df.columns if col.startswith("20")]  # 20191 ~ 20244 등

future_cols = ['20251', '20252', '20253', '20254']
future_data = []

X = np.arange(len(quarter_cols)).reshape(-1, 1)  # 예: [[0], [1], ..., [23]]

for _, row in leisure_income_df.iterrows():
    y = row[quarter_cols].values.astype(float)

    model = LinearRegression()
    model.fit(X, y)

    future_X = np.arange(len(quarter_cols), len(quarter_cols)+4).reshape(-1, 1)
    future_y = model.predict(future_X)

    future_data.append(future_y)

# 예측 결과 DataFrame으로 변환
future_df = pd.DataFrame(future_data, columns=future_cols)
future_df.insert(0, '행정동', leisure_income_df['행정동'])

# 기존 데이터와 합치기
full_df = pd.concat([leisure_income_df, future_df.drop(columns=['행정동'])], axis=1)
full_df


,행정동,20191,20192,20193,20194,20201,20202,20203,20204,20211,...,20241,20242,20243,20244,시설수,면적,20251,20252,20253,20254
0,가락1동,976.7,2742.5,5694.0,6619.9,5972.4,6075.7,6965.0,6100.9,7732.2,...,8140.3,23059.5,8857.3,20776.8,NaN,1.34,24651.008696,25535.125391,26419.242087,27303.358783
1,가락2동,24211.5,28670.5,31174.1,26322.9,22098.0,26554.3,24056.0,20526.6,18830.4,...,10294.5,31240.9,11523.5,29320.8,NaN,0.96,25670.742391,25644.971783,25619.201174,25593.430565
2,가락본동,80798.5,91568.5,87795.3,86659.8,67906.0,79581.1,60011.3,66875.0,62812.2,...,27053.3,85077.5,27520.1,81740.6,1089.0,1.13,69025.853623,68431.766580,67837.679536,67243.592493
3,가리봉동,3827.9,5010.0,5007.8,4413.4,4449.7,4455.0,4424.8,4513.2,4405.1,...,1888.1,5750.6,2074.7,6398.2,9.0,0.40,6450.265580,6461.434493,6472.603406,6483.772319
4,가산동,260854.5,296081.2,277126.2,283551.3,200981.8,235046.3,231374.4,290192.3,272056.3,...,351024.5,954394.8,354094.6,1073202.9,361.0,2.52,779536.484058,806195.840449,832855.196841,859514.553232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,효창동,5747.6,6618.2,7262.1,8126.6,4519.6,6966.1,5166.4,4298.1,4850.0,...,2021.5,5735.5,1384.6,3977.3,4.0,0.44,4823.021739,4743.753478,4664.485217,4585.216957
421,후암동,3234.9,3145.8,3577.2,3578.3,5075.8,17677.6,12930.8,12147.6,14079.5,...,2280.7,7518.6,3295.8,6704.8,16.0,0.86,12480.438043,12618.900087,12757.362130,12895.824174
422,휘경1동,19841.7,17889.2,16620.1,16791.3,14988.0,13030.4,11705.8,10332.0,11210.8,...,10112.8,28275.6,8734.3,27547.5,49.0,0.63,24630.279348,25076.107696,25521.936043,25967.764391
423,휘경2동,10747.5,11883.8,11683.4,13491.4,13793.7,12149.3,11239.8,21211.2,19423.0,...,8686.9,28600.8,10569.3,29147.7,NaN,1.05,27230.288406,27803.253145,28376.217884,28949.182623


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
X = np.arange(24).reshape(-1, 1)  # 24개 분기
y = leisure_income_df.loc[0, quarter_cols].values.astype(float)  # 특정 행정동 예시

# 훈련
model = LinearRegression()
model.fit(X, y)

# 예측
y_pred = model.predict(X)

# 평가 지표 계산
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"📊 회귀 평가 지표:")
print(f"✅ MSE  : {mse:.2f}")
print(f"✅ RMSE : {rmse:.2f}")
print(f"✅ MAE  : {mae:.2f}")
print(f"✅ R²   : {r2:.4f}")

📊 회귀 평가 지표:
✅ MSE  : 27924877.41
✅ RMSE : 5284.40
✅ MAE  : 3844.78
✅ R²   : 0.5729


In [38]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 225.6 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [39]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X, y)
y_pred = model.predict(X)

print("MSE:", mean_squared_error(y, y_pred))
print("R² :", r2_score(y, y_pred))

MSE: 109592.90731267481
R² : 0.9983237428292061


In [46]:
full_df['2025평균소득'] = full_df[['20251', '20252', '20253', '20254']].mean(axis=1)
discretizer = KBinsDiscretizer(n_bins=9, encode='ordinal', strategy='quantile')
full_df['2025소득분위'] = discretizer.fit_transform(full_df[['2025평균소득']]).astype(int) + 1
full_df

,행정동,20191,20192,20193,20194,20201,20202,20203,20204,20211,...,20243,20244,시설수,면적,20251,20252,20253,20254,2025평균소득,2025소득분위
0,가락1동,976.7,2742.5,5694.0,6619.9,5972.4,6075.7,6965.0,6100.9,7732.2,...,8857.3,20776.8,NaN,1.34,24651.008696,25535.125391,26419.242087,27303.358783,25977.183739,6
1,가락2동,24211.5,28670.5,31174.1,26322.9,22098.0,26554.3,24056.0,20526.6,18830.4,...,11523.5,29320.8,NaN,0.96,25670.742391,25644.971783,25619.201174,25593.430565,25632.086478,6
2,가락본동,80798.5,91568.5,87795.3,86659.8,67906.0,79581.1,60011.3,66875.0,62812.2,...,27520.1,81740.6,1089.0,1.13,69025.853623,68431.766580,67837.679536,67243.592493,68134.723058,8
3,가리봉동,3827.9,5010.0,5007.8,4413.4,4449.7,4455.0,4424.8,4513.2,4405.1,...,2074.7,6398.2,9.0,0.40,6450.265580,6461.434493,6472.603406,6483.772319,6467.018949,2
4,가산동,260854.5,296081.2,277126.2,283551.3,200981.8,235046.3,231374.4,290192.3,272056.3,...,354094.6,1073202.9,361.0,2.52,779536.484058,806195.840449,832855.196841,859514.553232,819525.518645,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,효창동,5747.6,6618.2,7262.1,8126.6,4519.6,6966.1,5166.4,4298.1,4850.0,...,1384.6,3977.3,4.0,0.44,4823.021739,4743.753478,4664.485217,4585.216957,4704.119348,1
421,후암동,3234.9,3145.8,3577.2,3578.3,5075.8,17677.6,12930.8,12147.6,14079.5,...,3295.8,6704.8,16.0,0.86,12480.438043,12618.900087,12757.362130,12895.824174,12688.131109,3
422,휘경1동,19841.7,17889.2,16620.1,16791.3,14988.0,13030.4,11705.8,10332.0,11210.8,...,8734.3,27547.5,49.0,0.63,24630.279348,25076.107696,25521.936043,25967.764391,25299.021870,5
423,휘경2동,10747.5,11883.8,11683.4,13491.4,13793.7,12149.3,11239.8,21211.2,19423.0,...,10569.3,29147.7,NaN,1.05,27230.288406,27803.253145,28376.217884,28949.182623,28089.735514,6


In [47]:
columns_to_merge = predicted_df[['행정동', '2025평균소득', '2025소득분위', '면적']]

# 기존 시설물 데이터 불러오기
facility_df = pd.read_csv("hangjung_facility.csv")

# 병합 (행정동 기준)
merged_df = facility_df.merge(columns_to_merge, on='행정동', how='left')

# 결과 저장
merged_df.to_csv("hangjung_facility_업데이트.csv", index=False, encoding='utf-8-sig')
print("✅ 'hangjung_facility_업데이트.csv' 저장 완료!")

NameError: name 'predicted_df' is not defined